In [79]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date,  datetime
%matplotlib inline

In [80]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion, make_pipeline, Pipeline

In [81]:
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

In [82]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable


In [83]:
!pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable


In [84]:
import xgboost as xgb
import catboost as cat
import lightgbm as light
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier

In [85]:
! pip install scikit-multilearn

Defaulting to user installation because normal site-packages is not writeable


In [86]:
!pip install community

Defaulting to user installation because normal site-packages is not writeable


In [87]:
!pip install networkx

Defaulting to user installation because normal site-packages is not writeable


In [88]:
# !pip install ipywidgets

In [89]:
from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset
from skmultilearn.cluster.networkx import NetworkXLabelGraphClusterer
from skmultilearn.cluster import LabelCooccurrenceGraphBuilder
from skmultilearn.ensemble import LabelSpacePartitioningClassifier

> import datasets

In [90]:
test_path = '../datasets/Test.csv'
train_path = '../datasets/Train.csv'
sample_submission_path = '../datasets/SampleSubmission.csv'

> define utitlity functions

load data

In [91]:
def load_data(path=""):
    data = pd.read_csv(path)
    return data

move columns

In [92]:
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()    
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

transform dates

In [93]:
def transform_dates(df, col_name='', calc_age=False, 
                    birth_year_col_name = '', drop_date=True):
    df[col_name] = pd.to_datetime(df[col_name])
    #fill empty date columns
    if df[col_name].isna().any() == True:
        fill_date = df[col_name].mode()[0]
        df[col_name] =  df[col_name].fillna(fill_date)
    
    #create new datelike features
    todays_date = date.today()
    todays_date = pd.to_datetime(todays_date)
    
    df['duration_till_date'] = (todays_date - df[col_name]).astype('str').apply(lambda x: x.split(' d'
                                                                             )[0].strip()).astype('int64')
    
    df['year'] = df[col_name].apply(lambda x: x.year)
    df['month'] = df[col_name].apply(lambda x: x.year)
    df['week_of_year'] = df[col_name].apply(lambda x: x.weekofyear)
    df['day_of_week'] = df[col_name].apply(lambda x: x.dayofweek)
    
    if calc_age:
        
        df['age'] = df['year'] - df[birth_year_col_name]
        
    
    if drop_date:
        df.drop(columns=[col_name], inplace=True)
        
    
    return df

In [94]:
def val_counts(df, columns=[]):
    for col in columns:
        print("**"*8)
        print(col)
        print("**"*8) 
        print(df[col].value_counts()) 

In [95]:
test_set = load_data(test_path)
train_set = load_data(train_path)
sample_submission = load_data(sample_submission_path)

In [96]:
train_set

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,1/2/2019,F,M,1987,1X1H,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,1/6/2019,F,M,1981,UAOD,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,1/6/2013,M,U,1991,748L,QZYX,90QI,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,1/8/2019,M,M,1990,1X1H,BP09,56SI,0,0,...,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,1/8/2019,M,M,1990,748L,NO3L,T4MS,0,0,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29127,8SBA93U,1/9/2019,F,M,1974,748L,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
29128,Y73ETTO,1/11/2018,M,M,1987,XX25,Q6J6,90QI,0,0,...,0,0,0,1,0,0,0,0,0,0
29129,MVXG2LC,1/2/2019,M,M,1985,BOAS,6PE7,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
29130,BUQ8DQN,1/7/2019,M,M,1989,UAOD,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0


In [97]:
test_set

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,F86J5PC,1/12/2018,M,M,1984,94KC,DZRV,90QI,0,0,...,0,0,0,0,0,0,0,0,0,0
1,H6141K3,1/10/2019,M,M,1996,1X1H,J9SY,90QI,0,0,...,0,0,0,1,0,0,0,0,0,0
2,RBAYUXZ,1/1/2020,F,W,1968,UAOD,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
3,KCBILBQ,1/2/2019,M,M,1989,94KC,2A7I,T4MS,0,0,...,0,0,0,0,0,0,0,0,0,0
4,LSEC1ZJ,1/2/2020,F,M,1982,UAOD,0KID,T4MS,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0GMU5UH,1/4/2019,F,M,1986,748L,2A7I,T4MS,0,0,...,0,0,0,0,1,0,1,0,0,0
9996,I9W11CD,1/8/2019,M,M,1980,UAOD,0KID,T4MS,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,42WTEGT,1/4/2018,M,M,1993,748L,SST3,56SI,0,0,...,0,0,0,1,0,0,0,0,0,0
9998,8EKC4O9,1/5/2015,F,U,1981,49BM,2A7I,T4MS,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
sample_submission

,ID X PCODE,Label
0,F86J5PC X P5DA,0
1,F86J5PC X RIBP,0
2,F86J5PC X 8NN1,0
3,F86J5PC X 7POT,0
4,F86J5PC X 66FJ,0
...,...,...
209995,WHII93K X JWFN,0
209996,WHII93K X JZ9D,0
209997,WHII93K X J9JW,0
209998,WHII93K X GHYX,0


### Data Cleaning

In [99]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29132 entries, 0 to 29131
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ID                        29132 non-null  object
 1   join_date                 29130 non-null  object
 2   sex                       29132 non-null  object
 3   marital_status            29132 non-null  object
 4   birth_year                29132 non-null  int64 
 5   branch_code               29132 non-null  object
 6   occupation_code           29132 non-null  object
 7   occupation_category_code  29132 non-null  object
 8   P5DA                      29132 non-null  int64 
 9   RIBP                      29132 non-null  int64 
 10  8NN1                      29132 non-null  int64 
 11  7POT                      29132 non-null  int64 
 12  66FJ                      29132 non-null  int64 
 13  GYSR                      29132 non-null  int64 
 14  SOP4                  

In [100]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ID                        10000 non-null  object
 1   join_date                 9999 non-null   object
 2   sex                       10000 non-null  object
 3   marital_status            10000 non-null  object
 4   birth_year                10000 non-null  int64 
 5   branch_code               10000 non-null  object
 6   occupation_code           10000 non-null  object
 7   occupation_category_code  10000 non-null  object
 8   P5DA                      10000 non-null  int64 
 9   RIBP                      10000 non-null  int64 
 10  8NN1                      10000 non-null  int64 
 11  7POT                      10000 non-null  int64 
 12  66FJ                      10000 non-null  int64 
 13  GYSR                      10000 non-null  int64 
 14  SOP4                   

> Check values in train columns

In [101]:
val_counts(train_set, columns=train_set.columns.to_list())

****************
ID
****************
1UPQVCF    1
ISOC4W4    1
0YG32ID    1
4WHL9TQ    1
P567Q0K    1
          ..
SIB3D8I    1
NIFNICW    1
7XXVCZR    1
T4OOKOP    1
B4XVVBE    1
Name: ID, Length: 29132, dtype: int64
****************
join_date
****************
1/5/2018      1319
1/5/2019      1199
1/4/2018      1123
1/4/2019      1043
1/1/2019       754
              ... 
22/10/2012       1
23/5/2013        1
8/9/2012         1
2/9/2012         1
11/1/2013        1
Name: join_date, Length: 132, dtype: int64
****************
sex
****************
M    20329
F     8803
Name: sex, dtype: int64
****************
marital_status
****************
M    22913
S     3267
U     2474
W      301
D      135
R       34
P        7
f        1
Name: marital_status, dtype: int64
****************
birth_year
****************
1983    1263
1984    1241
1982    1222
1980    1143
1986    1097
        ... 
1932       1
2011       1
1946       1
2005       1
1936       1
Name: birth_year, Length: 61, dtype: int64

> Check values in test columns

In [116]:
val_counts(test_set, columns=test_set.columns.to_list())

****************
ID
****************
UCFL73O    1
GPE8SKK    1
DHTZPAJ    1
7JDD83P    1
L5YJTP5    1
          ..
RM9X9AE    1
FYE72YU    1
G5VEYBG    1
21CM5Z0    1
JHEWY92    1
Name: ID, Length: 10000, dtype: int64
****************
age
****************
35    410
30    393
33    393
37    392
34    391
36    390
32    374
31    369
39    364
38    348
29    347
40    336
42    330
28    328
44    327
41    325
43    315
45    298
46    292
27    275
47    271
26    244
48    243
49    242
25    221
50    216
51    175
24    163
52    140
23    133
53    120
22    102
54     97
58     93
55     88
21     81
57     78
56     73
59     72
60     50
20     49
19     25
61     10
18      6
17      3
64      3
62      2
65      1
74      1
87      1
Name: age, dtype: int64
****************
duration_till_date
****************
973     514
608     395
974     369
609     336
607     289
       ... 
3895      2
2872      1
3892      1
2979      1
3130      1
Name: duration_till_date, Length: 1

> For every product codes, there is an `imbalance in the class`. lesser product patronage among the class

In [103]:
train_set.isna().any()

ID                          False
join_date                    True
sex                         False
marital_status              False
birth_year                  False
branch_code                 False
occupation_code             False
occupation_category_code    False
P5DA                        False
RIBP                        False
8NN1                        False
7POT                        False
66FJ                        False
GYSR                        False
SOP4                        False
RVSZ                        False
PYUQ                        False
LJR9                        False
N2MW                        False
AHXO                        False
BSTQ                        False
FM3X                        False
K6QO                        False
QBOL                        False
JWFN                        False
JZ9D                        False
J9JW                        False
GHYX                        False
ECY3                        False
dtype: bool

date as empty rows

In [104]:
test_set.isna().any()

ID                          False
join_date                    True
sex                         False
marital_status              False
birth_year                  False
branch_code                 False
occupation_code             False
occupation_category_code    False
P5DA                        False
RIBP                        False
8NN1                        False
7POT                        False
66FJ                        False
GYSR                        False
SOP4                        False
RVSZ                        False
PYUQ                        False
LJR9                        False
N2MW                        False
AHXO                        False
BSTQ                        False
FM3X                        False
K6QO                        False
QBOL                        False
JWFN                        False
JZ9D                        False
J9JW                        False
GHYX                        False
ECY3                        False
dtype: bool

no more null values in date

### Feature Enngineering

> feature engineer dates and time

In [105]:
train_set = transform_dates(train_set, col_name='join_date', calc_age=True, 
                            birth_year_col_name='birth_year') #drop_date=True) 

In [106]:
test_set = transform_dates(test_set, col_name='join_date', calc_age=True, 
                            birth_year_col_name='birth_year') #drop_date=True) 

In [107]:
train_set.head()

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,...,JZ9D,J9JW,GHYX,ECY3,duration_till_date,year,month,week_of_year,day_of_week,age
0,4WKQSBB,F,M,1987,1X1H,2A7I,T4MS,0,0,0,...,0,0,0,0,611,2019,2019,1,2,32
1,CP5S02H,F,M,1981,UAOD,2A7I,T4MS,0,0,0,...,0,0,0,0,607,2019,2019,1,6,38
2,2YKDILJ,M,U,1991,748L,QZYX,90QI,0,0,0,...,0,0,0,1,2798,2013,2013,1,6,22
3,2S9E81J,M,M,1990,1X1H,BP09,56SI,0,0,0,...,0,0,0,0,605,2019,2019,2,1,29
4,BHDYVFT,M,M,1990,748L,NO3L,T4MS,0,0,0,...,1,1,0,0,605,2019,2019,2,1,29


In [108]:
test_set.head()

,ID,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,8NN1,...,JZ9D,J9JW,GHYX,ECY3,duration_till_date,year,month,week_of_year,day_of_week,age
0,F86J5PC,M,M,1984,94KC,DZRV,90QI,0,0,0,...,0,0,0,0,966,2018,2018,2,4,34
1,H6141K3,M,M,1996,1X1H,J9SY,90QI,0,0,0,...,0,0,0,0,603,2019,2019,2,3,23
2,RBAYUXZ,F,W,1968,UAOD,2A7I,T4MS,0,0,0,...,0,0,0,0,247,2020,2020,1,2,52
3,KCBILBQ,M,M,1989,94KC,2A7I,T4MS,0,0,0,...,0,0,0,0,611,2019,2019,1,2,30
4,LSEC1ZJ,F,M,1982,UAOD,0KID,T4MS,0,0,0,...,1,0,0,0,246,2020,2020,1,3,38


In [109]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29132 entries, 0 to 29131
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ID                        29132 non-null  object
 1   sex                       29132 non-null  object
 2   marital_status            29132 non-null  object
 3   birth_year                29132 non-null  int64 
 4   branch_code               29132 non-null  object
 5   occupation_code           29132 non-null  object
 6   occupation_category_code  29132 non-null  object
 7   P5DA                      29132 non-null  int64 
 8   RIBP                      29132 non-null  int64 
 9   8NN1                      29132 non-null  int64 
 10  7POT                      29132 non-null  int64 
 11  66FJ                      29132 non-null  int64 
 12  GYSR                      29132 non-null  int64 
 13  SOP4                      29132 non-null  int64 
 14  RVSZ                  

In [110]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29132 entries, 0 to 29131
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ID                        29132 non-null  object
 1   sex                       29132 non-null  object
 2   marital_status            29132 non-null  object
 3   birth_year                29132 non-null  int64 
 4   branch_code               29132 non-null  object
 5   occupation_code           29132 non-null  object
 6   occupation_category_code  29132 non-null  object
 7   P5DA                      29132 non-null  int64 
 8   RIBP                      29132 non-null  int64 
 9   8NN1                      29132 non-null  int64 
 10  7POT                      29132 non-null  int64 
 11  66FJ                      29132 non-null  int64 
 12  GYSR                      29132 non-null  int64 
 13  SOP4                      29132 non-null  int64 
 14  RVSZ                  

> reorder columns

In [111]:
train_set = movecol(train_set, ['age','duration_till_date', 'year', 'month', 
                                'week_of_year', 'day_of_week'], ref_col='ID')

In [112]:
test_set = movecol(test_set, ['age','duration_till_date', 'year', 'month', 
                                'week_of_year', 'day_of_week'], ref_col='ID')

In [113]:
train_set.head()

,ID,age,duration_till_date,year,month,week_of_year,day_of_week,sex,marital_status,birth_year,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,32,611,2019,2019,1,2,F,M,1987,...,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,38,607,2019,2019,1,6,F,M,1981,...,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,22,2798,2013,2013,1,6,M,U,1991,...,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,29,605,2019,2019,2,1,M,M,1990,...,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,29,605,2019,2019,2,1,M,M,1990,...,0,0,0,0,0,0,1,1,0,0


In [114]:
test_set.head()

,ID,age,duration_till_date,year,month,week_of_year,day_of_week,sex,marital_status,birth_year,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,F86J5PC,34,966,2018,2018,2,4,M,M,1984,...,0,0,0,0,0,0,0,0,0,0
1,H6141K3,23,603,2019,2019,2,3,M,M,1996,...,0,0,0,1,0,0,0,0,0,0
2,RBAYUXZ,52,247,2020,2020,1,2,F,W,1968,...,0,0,0,1,0,0,0,0,0,0
3,KCBILBQ,30,611,2019,2019,1,2,M,M,1989,...,0,0,0,0,0,0,0,0,0,0
4,LSEC1ZJ,38,246,2020,2020,1,3,F,M,1982,...,0,0,0,0,0,0,1,0,0,0


In [115]:
train_set.columns

Index(['ID', 'age', 'duration_till_date', 'year', 'month', 'week_of_year',
       'day_of_week', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3'],
      dtype='object')

#### Further Feature Engineering

-  get product use frequency per user

In [117]:
train_product_ids  = train_set.columns.to_list()[13:]
train_product_ids

['P5DA',
 'RIBP',
 '8NN1',
 '7POT',
 '66FJ',
 'GYSR',
 'SOP4',
 'RVSZ',
 'PYUQ',
 'LJR9',
 'N2MW',
 'AHXO',
 'BSTQ',
 'FM3X',
 'K6QO',
 'QBOL',
 'JWFN',
 'JZ9D',
 'J9JW',
 'GHYX',
 'ECY3']

In [120]:
test_product_ids  = test_set.columns.to_list()[13:]
test_product_ids

['P5DA',
 'RIBP',
 '8NN1',
 '7POT',
 '66FJ',
 'GYSR',
 'SOP4',
 'RVSZ',
 'PYUQ',
 'LJR9',
 'N2MW',
 'AHXO',
 'BSTQ',
 'FM3X',
 'K6QO',
 'QBOL',
 'JWFN',
 'JZ9D',
 'J9JW',
 'GHYX',
 'ECY3']

In [121]:
f'lenght of insurance products {len(train_product_ids)}'

'lenght of insurance products 21'

In [122]:
non_product_cols = train_set.iloc[:, :12].columns.tolist()

In [123]:
non_product_cols

['ID',
 'age',
 'duration_till_date',
 'year',
 'month',
 'week_of_year',
 'day_of_week',
 'sex',
 'marital_status',
 'birth_year',
 'branch_code',
 'occupation_code']

In [124]:
train_set=train_set.melt(id_vars=non_product_cols, value_vars=train_product_ids, var_name='product_ids', value_name='labels')

In [125]:
test_set=test_set.melt(id_vars=non_product_cols, value_vars=test_product_ids, var_name='product_ids', value_name='labels')

In [127]:
train_set.head()

,ID,age,duration_till_date,year,month,week_of_year,day_of_week,sex,marital_status,birth_year,branch_code,occupation_code,product_ids,labels
0,4WKQSBB,32,611,2019,2019,1,2,F,M,1987,1X1H,2A7I,P5DA,0
1,CP5S02H,38,607,2019,2019,1,6,F,M,1981,UAOD,2A7I,P5DA,0
2,2YKDILJ,22,2798,2013,2013,1,6,M,U,1991,748L,QZYX,P5DA,0
3,2S9E81J,29,605,2019,2019,2,1,M,M,1990,1X1H,BP09,P5DA,0
4,BHDYVFT,29,605,2019,2019,2,1,M,M,1990,748L,NO3L,P5DA,0


In [128]:
test_set.head()

,ID,age,duration_till_date,year,month,week_of_year,day_of_week,sex,marital_status,birth_year,branch_code,occupation_code,product_ids,labels
0,F86J5PC,34,966,2018,2018,2,4,M,M,1984,94KC,DZRV,P5DA,0
1,H6141K3,23,603,2019,2019,2,3,M,M,1996,1X1H,J9SY,P5DA,0
2,RBAYUXZ,52,247,2020,2020,1,2,F,W,1968,UAOD,2A7I,P5DA,0
3,KCBILBQ,30,611,2019,2019,1,2,M,M,1989,94KC,2A7I,P5DA,0
4,LSEC1ZJ,38,246,2020,2020,1,3,F,M,1982,UAOD,0KID,P5DA,0


> sort dataset by customer ID and product id

In [129]:
train_set.sort_values(by=['ID','product_ids' ], inplace=True)

In [130]:
test_set.sort_values(by=['ID','product_ids' ], inplace=True)

> concatenate user ID and producct id

In [131]:
train_set['ID X PCODE'] = train_set.ID+' X '+train_set.product_ids

In [132]:
train_set['ID X PCODE']

135107    001IIBS X 66FJ
105975    001IIBS X 7POT
76843     001IIBS X 8NN1
339031    001IIBS X AHXO
368163    001IIBS X BSTQ
               ...      
260571    ZZZU5BE X PYUQ
464495    ZZZU5BE X QBOL
56647     ZZZU5BE X RIBP
231439    ZZZU5BE X RVSZ
202307    ZZZU5BE X SOP4
Name: ID X PCODE, Length: 611772, dtype: object

In [133]:
test_set['ID X PCODE'] = test_set.ID+' X '+train_set.product_ids
test_set['ID X PCODE']

49842     0021EE1 X RIBP
39842     0021EE1 X RIBP
29842     0021EE1 X RIBP
119842    0021EE1 X 66FJ
129842    0021EE1 X 66FJ
               ...      
86019     ZZYM5CT X 8NN1
156019    ZZYM5CT X GYSR
16019     ZZYM5CT X P5DA
76019     ZZYM5CT X 8NN1
66019     ZZYM5CT X 8NN1
Name: ID X PCODE, Length: 210000, dtype: object

In [137]:
train_set = movecol(train_set, cols_to_move=['ID X PCODE'], ref_col='ID')

In [138]:
test_set = movecol(test_set, cols_to_move=['ID X PCODE'], ref_col='ID')

<!-- > concatenate product ID and customer ID -->

In [344]:
train_set.head(2)

,ID,ID X PCODE,age,duration_till_date,year,month,week_of_year,day_of_week,sex,marital_status,birth_year,branch_code,occupation_code,product_ids,labels
135107,001IIBS,001IIBS X 66FJ,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,66FJ,0
105975,001IIBS,001IIBS X 7POT,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,7POT,0


In [142]:
test_ = train_set.copy()

In [145]:
test_

,ID,ID X PCODE,age,duration_till_date,year,month,week_of_year,day_of_week,sex,marital_status,birth_year,branch_code,occupation_code,product_ids,labels
135107,001IIBS,001IIBS X 66FJ,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,66FJ,0
105975,001IIBS,001IIBS X 7POT,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,7POT,0
76843,001IIBS,001IIBS X 8NN1,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,8NN1,0
339031,001IIBS,001IIBS X AHXO,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,AHXO,0
368163,001IIBS,001IIBS X BSTQ,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,BSTQ,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260571,ZZZU5BE,ZZZU5BE X PYUQ,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,PYUQ,0
464495,ZZZU5BE,ZZZU5BE X QBOL,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,QBOL,0
56647,ZZZU5BE,ZZZU5BE X RIBP,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,RIBP,0
231439,ZZZU5BE,ZZZU5BE X RVSZ,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,RVSZ,1


In [275]:
class_counts_per_products = pd.DataFrame(test_.groupby(['product_ids', 'labels'])['labels'].count()
            ).rename(columns={'labels':'vals'})

In [278]:
class_counts_per_products.head()

vals
product_ids labels       
66FJ        0       28793
            1         339
7POT        0       28816
            1         316
8NN1        0       28975

In [288]:
class_counts_per_products = class_counts_per_products.reset_index().pivot(index='product_ids', columns='labels')#.reset_index()

In [294]:
class_counts_per_products

vals       
labels           0      1
product_ids              
66FJ         28793    339
7POT         28816    316
8NN1         28975    157
AHXO         28593    539
BSTQ         28808    324
ECY3         28030   1102
FM3X         29022    110
GHYX         28230    902
GYSR         29128      4
J9JW         27714   1418
JWFN         28821    311
JZ9D         27707   1425
K6QO          7503  21629
LJR9         28778    354
N2MW         28294    838
P5DA         29092     40
PYUQ         26959   2173
QBOL         22299   6833
RIBP         27352   1780
RVSZ          3804  25328
SOP4         28701    431

In [302]:
class_counts_per_products = class_counts_per_products['vals']

In [325]:
class_counts_per_products = class_counts_per_products.reset_index().rename(columns={0:'no_purchase', 1:'purchase'})

In [327]:
class_counts_per_products.sort_values(by='purchase', ascending=False, inplace=True)

In [338]:
class_counts_per_products.columns.name = None

In [377]:
patronage_label = pd.DataFrame(class_counts_per_products.purchase.describe(percentiles=[
    .1,.25, .50, .75, 1])[['10%','25%','50%', '75%', '100%']]).reset_index().rename(columns={
    'index':'patronage', 'purchase':'value'})

In [400]:
patronage_label.rename(index={0:'very_poor', 1:'poor', 2:'avg', 3:'good', 4:'excellent'}, inplace=True)

In [401]:
patronage_label

,patronage,value
very_poor,10%,110.0
poor,25%,316.0
avg,50%,539.0
good,75%,1425.0
excellent,100%,25328.0


In [402]:
pat_dic = patronage_label.value.to_dict()
pat_dic 

{'very_poor': 110.0,
 'poor': 316.0,
 'avg': 539.0,
 'good': 1425.0,
 'excellent': 25328.0}

In [416]:
labels = []

for k, v in pat_dic.items():
    for i in class_counts_per_products.purchase.to_list():
        if i <= v:
            labels.append(k)
        else:
            pass
        

In [417]:
len(labels)

57

In [ ]:
class_counts_per_products['labels'] = 

> Defince function to label a product in relation to percentile

- 10% very poor

- 25% poor

- 50% average

- 75% good

- 100% excellent

In [391]:
def label_patronage(df, col_name=''):
    
    ratings = pd.DataFrame(df[col_name].describe(percentiles=[
    .1,.25, .50, .75, 1])[['10%','25%','50%', '75%', '100%']]).reset_index().rename(columns={
    'index':'patronage', 'purchase':'value'})
    
    
    

In [345]:
class_counts_per_products

,product_ids,no_purchase,purchase
19,RVSZ,3804,25328
12,K6QO,7503,21629
17,QBOL,22299,6833
16,PYUQ,26959,2173
18,RIBP,27352,1780
11,JZ9D,27707,1425
9,J9JW,27714,1418
5,ECY3,28030,1102
7,GHYX,28230,902
14,N2MW,28294,838


In [140]:
# val_counts(train_set, train_set.columns.to_list())

> Find ratio of class

In [837]:
train_set.labels.value_counts()

0    545419
1     66353
Name: labels, dtype: int64

The total number of subscriptions is more than no subscriptions -> imbalanced class

#### Prepare Baseline Model

In [843]:
base_model_data = train_set.drop(columns=['ID', 'labels'])

In [844]:
base_model_data

,age,duration_till_date,year,month,week_of_year,day_of_week,sex,marital_status,birth_year,branch_code,occupation_code,product_ids
135107,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,66FJ
105975,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,7POT
76843,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,8NN1
339031,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,AHXO
368163,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,BSTQ
...,...,...,...,...,...,...,...,...,...,...,...,...
260571,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,PYUQ
464495,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,QBOL
56647,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,RIBP
231439,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,RVSZ


> Create a Pipeline to preprocess categorical and numerical columns

In [848]:
base_cat = base_model_data.select_dtypes(include='object')
base_cat

,sex,marital_status,branch_code,occupation_code,product_ids
135107,F,M,9F9T,31JW,66FJ
105975,F,M,9F9T,31JW,7POT
76843,F,M,9F9T,31JW,8NN1
339031,F,M,9F9T,31JW,AHXO
368163,F,M,9F9T,31JW,BSTQ
...,...,...,...,...,...
260571,M,M,94KC,2A7I,PYUQ
464495,M,M,94KC,2A7I,QBOL
56647,M,M,94KC,2A7I,RIBP
231439,M,M,94KC,2A7I,RVSZ


In [849]:
base_num = base_model_data.select_dtypes(include='int64')
base_num

,age,duration_till_date,year,month,week_of_year,day_of_week,birth_year
135107,36,605,2019,2019,2,1,1983
105975,36,605,2019,2019,2,1,1983
76843,36,605,2019,2019,2,1,1983
339031,36,605,2019,2019,2,1,1983
368163,36,605,2019,2019,2,1,1983
...,...,...,...,...,...,...,...
260571,25,608,2019,2019,1,5,1994
464495,25,608,2019,2019,1,5,1994
56647,25,608,2019,2019,1,5,1994
231439,25,608,2019,2019,1,5,1994


In [ ]:
transformed_df = ColumnTransformer()

In [ ]:
create_dummies(base_model_data, columns= base_model_data.)

> Further Feature Engineering

In [838]:
train_set

,ID,age,duration_till_date,year,month,week_of_year,day_of_week,sex,marital_status,birth_year,branch_code,occupation_code,product_ids,labels
135107,001IIBS,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,66FJ,0
105975,001IIBS,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,7POT,0
76843,001IIBS,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,8NN1,0
339031,001IIBS,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,AHXO,0
368163,001IIBS,36,605,2019,2019,2,1,F,M,1983,9F9T,31JW,BSTQ,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260571,ZZZU5BE,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,PYUQ,0
464495,ZZZU5BE,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,QBOL,0
56647,ZZZU5BE,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,RIBP,0
231439,ZZZU5BE,25,608,2019,2019,1,5,M,M,1994,94KC,2A7I,RVSZ,1


In [840]:
train_set.groupby('product_ids')['ID'].count() 

product_ids
66FJ    29132
7POT    29132
8NN1    29132
AHXO    29132
BSTQ    29132
ECY3    29132
FM3X    29132
GHYX    29132
GYSR    29132
J9JW    29132
JWFN    29132
JZ9D    29132
K6QO    29132
LJR9    29132
N2MW    29132
P5DA    29132
PYUQ    29132
QBOL    29132
RIBP    29132
RVSZ    29132
SOP4    29132
Name: ID, dtype: int64

#### Modelling